In [1]:
%%capture
!pip install kaggle
!pip install efficientnet_pytorch

In [2]:
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

from zipfile import ZipFile
dataset_path = './dogs-vs-cats-redux-kernels-edition.zip'

with ZipFile(dataset_path, 'r') as z:
    z.extractall(path='./dataset')
    print("Done")

train_dataset_path = "./dataset/train.zip"
with ZipFile(train_dataset_path, 'r') as z:
    z.extractall(path='./dataset/train')
    print("Done")

test_dataset_path = "./dataset/test.zip"
with ZipFile(test_dataset_path, 'r') as z:
    z.extractall(path='./dataset/test')
    print("Done")

Saving kaggle.json to kaggle.json
100% 811M/814M [00:37<00:00, 24.9MB/s]
100% 814M/814M [00:37<00:00, 22.5MB/s]
Done
Done
Done


In [3]:
import os
import re
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from sklearn import metrics

import torch
from torch import nn, optim
import torch.nn.functional as F
from efficientnet_pytorch import EfficientNet

import albumentations as A
from albumentations.pytorch import ToTensorV2

In [4]:
class CatDogDataset:
    def __init__(self, img_paths, labels, transform=None):
        self.img_paths = img_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, index):
        img_path = self.img_paths[index]
        img = np.array(Image.open(img_path))
        label = self.labels[index]

        if self.transform is not None:
            img = self.transform(image=img)["image"]

        return img, torch.tensor(label, dtype=torch.float)

In [21]:
def get_target(img_name):
    target_name = img_name.split(".")[0]
    if target_name == "dog": return 1
    elif target_name == "cat": return 0 
    return -1

IMGS_DIR = "/content/dataset/train/train"

img_names = os.listdir(IMGS_DIR)
img_names = list(sorted(img_names, key=lambda x: int(re.findall(r"\d+", x)[0])))
df = pd.DataFrame(img_names, columns=["img_names"])
df["labels"] = df.img_names.apply(lambda img_name: get_target(img_name))

df_train = df.head(20000).copy().reset_index(drop=True)
df_test = df.tail(5000).copy().reset_index(drop=True)

In [29]:
HEIGHT = 128
WIDTH = 128
NUM_WORKERS = 0
TRAIN_BATCH_SIZE = 32
TEST_BATCH_SIZE = 32

transform = A.Compose(
    [
        A.Resize(height=HEIGHT, width=WIDTH),
        A.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ]
)
train_img_names = df_train.img_names.values.tolist()
train_img_paths = [os.path.join(IMGS_DIR, train_img_name) for train_img_name in train_img_names]
train_labels = df_train.labels.values

train_dataset = CatDogDataset(train_img_paths, train_labels, transform)
train_data_loader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
)

test_img_names = df_test.img_names.values.tolist()
test_img_paths = [os.path.join(IMGS_DIR, test_img_name) for test_img_name in test_img_names]
test_labels = df_test.labels.values

test_dataset = CatDogDataset(test_img_paths, test_labels, transform)
test_data_loader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=TEST_BATCH_SIZE,
    shuffle=False,
)

In [40]:
def train(train_data_loader, model, bce, optimizer, scaler, epoch):
    model.train()
    losses = []
    loop = tqdm(train_data_loader, total=len(train_data_loader), leave=True)
    for batch_idx, (imgs, labels) in enumerate(loop):
        imgs = imgs.to(DEVICE)
        labels = labels.unsqueeze(1).to(DEVICE)

        with torch.cuda.amp.autocast():
            outputs = model(imgs)
            loss = bce(outputs, labels)

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        losses.append(loss.item())
        loop.set_description(f"Epoch [{epoch}/{EPOCHS-1}]")
        loop.set_postfix(loss=loss.item())
    loop.close()
    return sum(losses) / len(losses)

def evaluate(test_data_loader, model):
    model.eval()
    predictions = []
    targets = []

    for imgs, labels in tqdm(test_data_loader):
        imgs = imgs.to(device=DEVICE)
        labels = labels.unsqueeze(1).to(DEVICE)

        with torch.no_grad():
            outputs = model(imgs)

        predictions.append(torch.sigmoid(outputs).detach().cpu())
        targets.append(labels.detach().cpu())

    predictions = torch.cat(predictions, dim=0).view(-1).numpy()
    targets = torch.cat(targets, dim=0).view(-1).numpy()

    accuracy = metrics.accuracy_score(targets, predictions > 0.5)
    log_loss = metrics.log_loss(targets, predictions)
    return accuracy, log_loss

In [41]:
def save_checkpoint(model, optimizer, filename="model.pth.tar"):
    print("=>Saving Checkpoint...")
    checkpoint = {"state_dict": model.state_dict(), "optimizer": optimizer.state_dict()}
    torch.save(checkpoint, filename)

def load_checkpoint(checkpoint_path, model, optimizer):
    print("=>Loading Checkpoint...")
    checkpoint = torch.load(checkpoint_path, map_location=torch.device(DEVICE))
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

In [42]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LOAD_MODEL = False
SAVE_MODEL = False
CHECKPOINT_FILE = "model.pth.tar"
WEIGHT_DECAY = 1e-4
LEARNING_RATE = 1e-4
EPOCHS = 3

model = EfficientNet.from_pretrained("efficientnet-b7")
model._fc = nn.Linear(2560, 1)
model = model.to(DEVICE)

bce = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scaler = torch.cuda.amp.GradScaler()

if LOAD_MODEL:
    load_checkpoint(CHECKPOINT_FILE, model, optimizer)

for epoch in range(EPOCHS):
    loss = train(train_data_loader, model, bce, optimizer, scaler, epoch)
    accuracy, log_loss = evaluate(test_data_loader, model)
    save_checkpoint(model, optimizer, filename=CHECKPOINT_FILE)

    print(f"EPOCH: {epoch}, loss: {loss}, accuracy: {accuracy}, log_loss: {log_loss}")

Loaded pretrained weights for efficientnet-b7


100%|██████████| 157/157 [00:38<00:00,  4.12it/s]


=>Saving Checkpoint...
EPOCH: 0, loss: 0.16170488437861205, accuracy: 0.9714, log_loss: 0.07653108054083904


100%|██████████| 157/157 [00:38<00:00,  4.10it/s]


=>Saving Checkpoint...
EPOCH: 1, loss: 0.03348504933305085, accuracy: 0.9724, log_loss: 0.07896189737231597


100%|██████████| 157/157 [00:37<00:00,  4.15it/s]


=>Saving Checkpoint...
EPOCH: 2, loss: 0.014589819519873709, accuracy: 0.9756, log_loss: 0.07983680439214447


In [43]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [46]:
import shutil
os.mkdir("model")
gdrive_path = "/content/gdrive/MyDrive/googleColab/EfficientNetb7"
shutil.make_archive(os.path.join(gdrive_path, "model"), 'zip', "model")

'/content/gdrive/MyDrive/googleColab/EfficientNetb7/model.zip'

In [47]:
with ZipFile(os.path.join(gdrive_path, "model.zip"), 'r') as z:
    z.extractall(path=gdrive_path)
    print("Done")

Done
